In [1]:
import warnings

warnings.filterwarnings("ignore")

from datasets import load_dataset

data = load_dataset("glue", "mnli")
data

DatasetDict({
    train: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 392702
    })
    validation_matched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9815
    })
    validation_mismatched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9832
    })
    test_matched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9796
    })
    test_mismatched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9847
    })
})

In [2]:
from transformers import AutoTokenizer

checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [3]:
def tokenize_fun(examples):
    return tokenizer(examples["premise"], examples["hypothesis"], truncation=True)

In [4]:
tokenizer_datasets = data.map(tokenize_fun, batched=True)
tokenizer_datasets

DatasetDict({
    train: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 392702
    })
    validation_matched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 9815
    })
    validation_mismatched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 9832
    })
    test_matched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 9796
    })
    test_mismatched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 9847
    })
})

In [5]:
#对数据集进行填充操作
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [6]:
from peft import LoraConfig, TaskType
peft_config = LoraConfig(task_type=TaskType.SEQ_CLS, inference_mode=False, r=8, lora_alpha=32, lora_dropout=0.1)


In [14]:
#训练参数
from transformers import TrainingArguments
from transformers import AutoModelForSequenceClassification
from transformers import Trainer
import numpy as np
from datasets import load_metric
from peft import LoraConfig, TaskType
from peft import get_peft_model

metric = load_metric("accuracy")

def compute_metrics(eval_preds): 
    predictions, labels = eval_preds
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)
training_args = TrainingArguments("out_files",
                                  per_device_train_batch_size=16,
                                  num_train_epochs=1,
                                  logging_steps=50,
                                  evaluation_strategy='no')
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=3)

peft_config = LoraConfig(task_type=TaskType.SEQ_CLS, inference_mode=False, r=8, lora_alpha=32, lora_dropout=0.1)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenizer_datasets["train"],
    eval_dataset=tokenizer_datasets["validation_matched"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)
trainer.train()

Using the latest cached version of the module from C:\Users\admin\.cache\huggingface\modules\datasets_modules\metrics\accuracy\9756d5fa4a0f9da966341741fc3926eafdc604b8276add51d5abbaa8958a25f9 (last modified on Mon May 13 15:29:26 2024) since it couldn't be found locally at accuracy, or remotely on the Hugging Face Hub.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 297,219 || all params: 109,781,766 || trainable%: 0.27073621679578375


Step,Training Loss
50,1.123900
100,1.110000
150,1.104000
200,1.097700
250,1.088500
300,1.066800
350,1.047300
400,1.023600
450,1.002400


Step,Training Loss
50,1.123900
100,1.110000
150,1.104000
200,1.097700
250,1.088500
300,1.066800
350,1.047300
400,1.023600
450,1.002400
500,1.002300


Step,Training Loss
50,1.123900
100,1.110000
150,1.104000
200,1.097700
250,1.088500
300,1.066800
350,1.047300
400,1.023600
450,1.002400
500,1.002300


...

Step,Training Loss
50,1.123900
100,1.110000
150,1.104000
200,1.097700
250,1.088500
300,1.066800
350,1.047300
400,1.023600
450,1.002400
500,1.002300


Step,Training Loss
50,1.123900
100,1.110000
150,1.104000
200,1.097700
250,1.088500
300,1.066800
350,1.047300
400,1.023600
450,1.002400
500,1.002300


Step,Training Loss
50,1.123900
100,1.110000
150,1.104000
200,1.097700
250,1.088500
300,1.066800
350,1.047300
400,1.023600
450,1.002400
500,1.002300


Step,Training Loss
50,1.123900
100,1.110000
150,1.104000
200,1.097700
250,1.088500
300,1.066800
350,1.047300
400,1.023600
450,1.002400
500,1.002300


KeyboardInterrupt: 

In [ ]:
# from transformers import Trainer
# 
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=tokenizer_datasets["train"],  #取前20000个样本
#     eval_dataset=tokenizer_datasets["validation_matched"],
#     data_collator=data_collator,
#     tokenizer=tokenizer,
#     compute_metrics=compute_metrics
# )
# 
# 


In [ ]:
# trainer.train()

In [16]:
predictions = trainer.predict(tokenizer_datasets["validation_matched"].select(range(1000)))
print(predictions.predictions.shape, predictions.label_ids.shape)

Step,Training Loss
50,1.123900
100,1.110000
150,1.104000
200,1.097700
250,1.088500
300,1.066800
350,1.047300
400,1.023600
450,1.002400
500,1.002300


(1000, 3) (1000,)


In [17]:
metric = load_metric("glue", "mnli")
preds = np.argmax(predictions.predictions, axis=1)
metric.compute(predictions=preds, references=predictions.label_ids)

{'accuracy': 0.71}

In [18]:
print(predictions.predictions)

[[-1.0716455   1.558393   -0.7187973 ]
 [-2.51354    -0.8897644   1.5188805 ]
 [-0.5514954   0.92744076 -0.30621362]
 ...
 [ 1.4753689  -0.13601232 -0.40316856]
 [-0.5112553   1.2894444  -0.6917718 ]
 [ 0.5733509   0.5475807  -0.51116425]]
